# Métodos no supervisados: Topic Modeling y Clustering
Cuándo se está trabajando con un dataset compuesto de una gran cantidad de documentos se quiere saber cuál es el tema del que tratan. Es muy complicado asignar un único tema al corpus debido a qué está compuesto por textos con gran variedad de temas, más en este caso en el que se trabaja con un dataset conformado por comentarios de usuarios. 

Es en este punto cuando entra en juego el Topic Modeling (Modelado de temas). Este tipo de modelado se utiliza para determinar la estructura global del corpus mediante diversas técnicas estadísticas, no para asignar un tema concreto a cada uno de los documentos del corpus.

En este apartado se estudiarán varios métodos para el modelado de temas que permitirán entender su funcionamiento y cómo pueden ser utilizados para generar resúmenes de los documentos de forma rápida.

Para esta tarea se utilizará el dataset conformado por los comentarios de usuarios del repositorio "Zigbee2mqtt" que ya ha sido utilizado en tareas anteriores, lo que permite agilizar el proceso de modelado de temas porque este ya está tokenizado, vectorizado, etc.

Al igual que en todos los apartados, se comienza importando los ajustes del proyecto junto con la base de datos en la que se encuentra almacenado el Data Frame.

In [2]:
import sys, os

#Carga del archivo setup.py
%run -i ../pyenv_settings/setup.py

#Imports y configuraciones de gráficas
%run "$BASE_DIR/pyenv_settings/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# # to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# # otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

You are working on a local system.
Files will be searched relative to "..".


In [3]:
#Conexión con la base de datos en la que tenemos guardado el Data Frame
db_name = "../data/zigbee2mqtt_comments.db"
con = sqlite3.connect(db_name)
df = pd.read_sql("select * from comments", con)
con.close()

#Comprobación de que se ha cargado correctamente
print(df.columns)
print(df[['normalized_text', 'tokens']].head(4))

Index(['id', 'user', 'text', 'impurity', 'clean_text', 'normalized_text',
       'tokens'],
      dtype='object')
                                                                                                                                                                                           normalized_text  \
0                                                                    This issue is stale because it has been open 30 days with no activity. Remove stale label or comment or this will be closed in 7 days   
1  Also, after updating the z2m, cyclic reboots began ''' Starting Zigbee2MQTT without watchdog. INFO: Preparing to start... INFO: Socat not enabled INFO: Starting Zigbee2MQTT... Starting Zigbee2MQTT...   
2  Hi ! Since 2 or 3 days, MQTT suddenly fail. A few messages in the log, many auto restart, and works again ... Very strange. In the log INFO: Preparing to start... ERROR: Got unexpected response fr...   
3  I don't know if it's exactly the same, but since v1.42 I ha

**Recordar que en la columna "normalized_text" se encuentran todos los comentarios ya normalizados y vectorizados, es decir, con todos los pasos que se han seguido para obtener un texto limpio y listo para utilizarse en tareas de modelado**

## Pasos previos
Antes de comenzar el modelado, es recomendable conocer la información del corpues para así determinar cuáles son las entidades que se analizarán.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2678 entries, 0 to 2677
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2678 non-null   int64  
 1   user             2678 non-null   object 
 2   text             2678 non-null   object 
 3   impurity         2678 non-null   float64
 4   clean_text       2678 non-null   object 
 5   normalized_text  2678 non-null   object 
 6   tokens           2678 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 146.6+ KB


A priori parece una buena base para el modelado teniendo en cuenta que no hay elemenos nulos en ninguna columna. De todos modos, se puede imprimir por pantalla alguna muestra de estos tectos para comprobar si estos contienen caracteres especiales como puede ser la tabulación (\t), nueva línea (\r), retorno de carro (\r), etc.

In [8]:
print(repr(df.iloc[0]["normalized_text"][0:200]))
print(repr(df.iloc[-1]["normalized_text"][0:200]))

'This issue is stale because it has been open 30 days with no activity. Remove stale label or comment or this will be closed in 7 days'
"Is thist error? I have (old config) ''' socat: restartdelay: 1 initialdelay: 1 ''' in config but when i check default config this positions will disaapear."


Como el data frame ya había sido procesado y limpiado previamente, no se encuentran caracteres especiales que puedan dificultar el modelado.

Una vez completado este paso, se pasaría a la vectorización de los datos. En este caso, los datos ya fueron vectorizados anteriormente en *e_featureEngineering_and_syntactSimilarity.ipynb* de este mismo proyecto, permitiendo ahorrar tiempo y agilizar el proceso para cumplir el objetivo de este apartado.

## Factorización de matrices no negativas (NMF)
Conceptualmente, la forma más sencilla de hallar la estructura implícita en el corpus es la factorización de la matriz de términos, pero puede presentar un gasto computacional muy elevado. 

En lugar de esto, se puede realizar una factorización aproximada que es menos costoso y al mismo tiempo arroja buenos resultados.

Algunos métodos de álgebra lineal permiten representar la matriz como el producto de otras dos matrices no negativas. En este caso se nombrará la matriz original como *V*, y los factores *W* y *H*. 

## Creación de Modelos temáticos usando NMF
Casi todos los modelados de temas necesitan un número de temas como parámetro de entrada. En lugar de utilizar todos los temas de todos los textos, se utilizará un número aleatorio para esta tarea, por ejemplo 10. Este número es variable en cualquier caso, al fin y al cabo lo que se busca es un resultado más afinado, pero tampoco puede ser un número demasiado elevado de forma que el gasto computacional exceda los valores deseados.

Como la vectorización de los textos fue llevada a cabo en otro documento del proyecto, en este se deberá realizar el proceso otra vez para poder hacer uso de la variable que almacena los vectores.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

tfidf_text_vectorizer = TfidfVectorizer(stop_words=list(stopwords), min_df=5, max_df=0.7)
tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(df['normalized_text'])
tfidf_text_vectors.shape

(2678, 1721)

In [25]:
# Comprobación de que no se pierde información
df['normalized_text'].info()
print(df['normalized_text'].sample(5))

<class 'pandas.core.series.Series'>
RangeIndex: 2678 entries, 0 to 2677
Series name: normalized_text
Non-Null Count  Dtype 
--------------  ----- 
2678 non-null   object
dtypes: object(1)
memory usage: 21.0+ KB
1200                                                            This issue is stale because it has been open 30 days with no activity. Remove stale label or comment or this will be closed in 7 days
1258                                                                                                                                                                                           !image
172                                                                                                                                                 Hi. I have the same problem. Is there a solution?
2303                                                                                                                                       Same issue on latest HA with Zigbee2mqtt version: 1.18.1

In [36]:
from sklearn.decomposition import NMF

nmf_text_model = NMF(n_components=10, random_state=42)
W_text_matrix = nmf_text_model.fit_transform(tfidf_text_vectors)
H_text_matrix = nmf_text_model.components_

El tema de un texto viene dado por la distribución de las palabras que contiene, por lo tanto, analizar esta distribución puede ser de gran ayuda para descubrir los temas.

Haciendo uso de la matriz H, se debe encontrar el índice de los mayores valores de cada fila para luego ser utilizado como índice de búsqueda en el vocabulario.

Se definirá una función que muestre por pantalla un resumen de los temas (topics) que NMF ha detectado en los textos.

In [37]:
def display_topics(model, features, no_top_words=5):
    for topic, words in enumerate(model.components_):
        total = words.sum()
        largest = words.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print("  %s (%2.2f)" % (features[largest[i]], abs(words[largest[i]]*100.0/total)))

In [38]:
display_topics(nmf_text_model, tfidf_text_vectorizer.get_feature_names_out())


Topic 00
  stale (18.06)
  days (17.31)
  activity (8.96)
  label (8.96)
  comment (8.86)

Topic 01
  add (2.32)
  z2m (2.01)
  version (1.86)
  ha (1.43)
  new (1.23)

Topic 02
  thanks (50.82)
  worked (3.38)
  working (1.87)
  lot (1.09)
  reply (0.99)

Topic 03
  issue (41.26)
  having (2.15)
  got (1.62)
  close (1.11)
  exact (1.07)

Topic 04
  _url_ (29.25)
  duplicate (4.55)
  related (2.59)
  closing (1.28)
  device (1.22)

Topic 05
  error (3.03)
  zigbee (2.55)
  zigbee2mqtt (2.52)
  herdsman (2.11)
  info (2.04)

Topic 06
  problem (21.86)
  solution (3.05)
  solved (2.78)
  zha (1.10)
  thank (1.09)

Topic 07
  config (4.15)
  zigbee2mqtt (2.28)
  configuration (2.24)
  yaml (2.03)
  addon (1.85)

Topic 08
  fixed (16.32)
  edge (5.29)
  dev (4.58)
  branch (4.31)
  latest (3.39)

Topic 09
  update (13.43)
  42 (3.94)
  working (1.91)
  version (1.40)
  updated (1.36)


La salida muestra las palabras más relevantes para cada tema, permitiendo determinar la temática de algunos de los comentarios. Por ejemplo, en *Topic 02* se puede deducir que se trata de un mensaje de agradeciemiento como respuesta a otro y en *Topic 04* que se trata de algún problema con una URL concreta.

Sería interesante conocer como de grande es una temática, es decir, cuantos comentarios están relacionados con un mismo tema por ejemplo. 

Esto se puede calcular utilizando la matriz de temas de un documento y sumando las contribuciones individuales a este a lo largo de todos los documentos del data frame.

In [39]:
W_text_matrix.sum(axis=0)/W_text_matrix.sum()*100.0

array([18.15077823, 11.61180167,  5.36082748,  9.99092406,  8.43365604,
       10.84861004,  7.45875981, 12.96782206,  6.9400853 ,  8.23673532])

Este resultado indica que hay temas de mayor y menor peso pero no hay grandes diferencias en los porcentajes, indicando una supuesta buena calidad al tener una distribución bastante similar.